# La data visualisation avec Python

## Les graphiques interactifs avec d’autres packages et outils

### Création d’une visualisation avec Bokeh

Il s’agit ici de construire une visualisation qui sera disponible au format html, codée
en JavaScript grâce à BokehJS. Bokeh va générer ce fichier à partir de votre code en
Python.

Si nous nous intéressons aux données AirBnB :

In [1]:
import pandas as pd

In [2]:
# on importe les données
listing=pd.read_csv("./data/airbnb.csv")
listing["price"]=pd.to_numeric(listing["price"].str.replace("$","")\
                               .str.replace(",",""))

/Users/ldocao/.pyenv/versions/3.8-dev/envs/python-gis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show, output_file
# on crée les données
listing_chers = listing[listing["price"]>1000][["price","name","room_type","bedrooms"]]
# on définit le titre
TITLE = "Les logements les plus chers de Paris"
# on définit les outils que l’on veut afficher
tools = "pan, wheel_zoom, box_zoom, reset, save".split(',')
# on définit les informations devant apparaître lorsqu’on survole un point
hover = HoverTool(tooltips=[("Price", "@price"),
                            ("Description:", "@name"),
                            ("Type de logement:", "@room_type"),
                            ("Nombre de chambres:","@bedrooms")
                           ])
tools.append(hover)
# on crée le graphique et on définit les axes
p = figure(tools=tools, toolbar_location="above",  plot_width=1200, title=TITLE)
p.xaxis.axis_label = "Prix"
p.yaxis.axis_label = "Nombre de chambres"

# on définit les données (un dictionnaire est attendu mais un DataFrame fonctionne)
source = ColumnDataSource(listing_chers)

# on ajoute les points sous forme de cercles
p.circle("price", "bedrooms", size=5, source=source,
line_color="black", fill_alpha=0.8)

# on sauvegarde le fichier html
output_file("logements-bokeh.html", title="AirBnB à Paris")

# on ouvre un onglet du navigateur pour afficher le résultat
show(p)

On a donc extrait un DataFrame avec les logements à plus de 1000 euros et on
les a représentés dans un graphique interactif.

Les graphiques Bokeh peuvent ensuite très facilement s’intégrer dans des pages
web plus évoluées notamment grâce à l’environnement web Flask de Python.

### Création d’une application web avec Bokeh

La représentation précédente permet de visualiser des données, mais celles-ci sont figées et stockées dans le fichier html créé. Bien souvent, on voudra aller plus loin et créer une application interactive sur les données.

Pour cela, on va utiliser Bokeh et on va concevoir un fichier Python avec l’extension .py qui va inclure notre application. La partie application basée sur un serveur de Bokeh se base sur un environnement web nommé Tornado.

La documentation de Bokeh comprend de nombreuses informations :
https://bokeh.pydata.org/en/latest/

L’exemple donné ici est un exemple simple avec lancement de l’application en
local. Nous allons donc créer un fichier appli.py, et une fois ce fichier complété, nous pourrons l’utiliser pour lancer notre application avec Bokeh.
Le code est dans un seul fichier avec la forme suivante :

In [5]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, Div, HoverTool
from bokeh.models.widgets import Select
from bokeh.io import curdoc

# Récupération et préparation des données
# (on extrait uniquement les logements avec plus de 20 commentaires)
listing=pd.read_csv("./data/airbnb.csv", low_memory=False)
listing_chers = listing[listing["number_of_reviews"]>20]\
[["host_is_superhost","number_of_reviews", "price","name","room_type",
  "bedrooms","review_scores_rating"]]

# Définition des widgets (un outil de sélection en fonction de la colonne
# superhost)
superhost = Select(title="Super-host", value="All", options=["Vrai","Faux"])

# Définition de la source de données. Elle est vide et utilise un dictionnaire
source = ColumnDataSource(data=dict(nb_com=[], note_com=[], type_chambre=[],
                                    name=[], price=[]))
# Définition des informations à afficher lorsqu’on passe sur un point
TOOLTIPS=HoverTool(tooltips=[
    ("Nom", "@name"),
    ("Prix", "@price"),
    ("Nombre de commentaires", "@nb_com"),
    ("Note moyenne", "@note_com"),
    ("Type logement", "@type_chambre")
])

# construction de la figure et ajout des points à partir des données
p = figure(plot_height=600, plot_width=700,
           title="", toolbar_location=None, tools=[TOOLTIPS])
p.circle(x="nb_com", y="note_com", source=source, size=2)

# définition d’une fonction de mise à jour des données
def update() :
    if superhost.value == "Vrai":
        super_h="t"
    else:
        super_h="f"

    listing2=listing_chers[listing_chers["host_is_superhost"]==super_h]
    p.xaxis.axis_label = "Nombre de commentaires"
    p.yaxis.axis_label = "Note moyenne"
    # mise à jour des données
    source.data = dict(nb_com=listing2["number_of_reviews"],
                       note_com=listing2["review_scores_rating"],
                       type_chambre = listing2["room_type"],
                       name=listing2["name"],
                       price=listing2["price"]
                      )
# gestion des contrôles pour la mise à jour
# (on en a un seul dans notre cas Select)
controls = [superhost]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

# construction du layout pour l’affichage
inputs = widgetbox(*controls, sizing_mode="fixed")
l = layout([inputs, p], sizing_mode="fixed")

# premier chargement des données
update()

# utilisation de curdoc() pour générer la dataviz
curdoc().add_root(l)
curdoc().title = "AirBnB"

Nous utilisons dans l’invite de commandes générale, ou dans celle d’Anaconda, la
commande suivante :

L’application est donc lancée sur le serveur Bokeh en local et vous pouvez y accéder en utilisant le lien : 

http://localhost:5006/appli

Cette application web permet de croiser le nombre de commentaires et l’évaluation moyenne des logements. Une liste déroulante offre la possibilité d’afficher les logements ayant le label super-host ou non. Par ailleurs, lorsqu’on passe sur chaque point les caractéristiques du logement apparaissent.

Le package **Dash** offre une solution alternative récente qui vous permet de construire des applications web en utilisant l’environnement de Plotly. Bokeh et Dash sont aujourd’hui des solutions en pleine évolution.

In [ ]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Sample DataFrame
# On récupère les données
df2 = pd.read_csv("./data/effectifs-premier-degre.csv")
df2.Annee_scolaire = pd.to_numeric(df2.Annee_scolaire.str[:4])
df3 = df2.groupby(["Annee_scolaire","Libellé département"], as_index=False).agg({"Nombre_d_eleves":"sum", "Type d'établissement" : "count", "Academie":"first"})

# Dash app - The CSS code is pulled in from an external file
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# This defines the HTML layout
app.layout = html.Div([
    html.H1('My Report'),
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='year-slider',
        min=df3['Annee_scolaire'].min(),
        max=df3['Annee_scolaire'].max(),
        value=df3['Annee_scolaire'].min(),
        marks={str(year): str(year) for year in df3['Annee_scolaire'].unique()},
        step=None
    )
])

# This code runs every time the slider below the chart is changed
@app.callback(Output('graph-with-slider', 'figure'), [Input('year-slider', 'value')])
def update_figure(selected_year):
    filtered_df = df3[df3.Annee_scolaire == selected_year]
    traces = []
    for i in filtered_df.Academie.unique():
        df_by_continent = filtered_df[filtered_df['Academie'] == i]
        traces.append(dict(
            x=df_by_continent['Nombre_d_eleves'],
            y=df_by_continent["Type d'établissement"],
            text=df_by_continent['Libellé département'],
            mode='markers',
            opacity=0.7,
            marker={'size': 15, 'line': {'width': 0.5, 'color': 'white'}},
            name=i
        ))

    return {
        'data': traces,
        'layout': dict(
            xaxis={'title': "Nombre d'élèves", 'range': [0, df3["Nombre_d_eleves"].max()]},
            yaxis={'title': "Nombre d'établissements", 'range': [0, df3["Type d'établissement"].max()]},
            margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
            legend={'x': 0, 'y': 1},
            hovermode='closest',
            transition={'duration': 500},
        )
    }

if __name__ == '__main__':
    app.run_server(debug=False)

**Exercice :** Essayez d'adapté cet exemple aux données Covid